In [ ]:
#|default_exp n1d_embedding_analysis
import numpy as np
import matplotlib.pyplot as plt
import deepdish

import os
os.environ["GEOMSTATS_BACKEND"] = "pytorch"

# models
import torch
from autometric.autoencoders import DistanceMatchingAutoencoder
from autometric.datasets import *

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

%load_ext autoreload
%autoreload 2

# 1d Embedding Analysis

What can the autometrics tell us? This goes to the heart of the project - and, in this notebook, we develop the tools based on the metrics.

## Eigenvalues of the Metric

Each eigenvalue represents the degree of preservation applied to each direction in the ambient space under the embedding, where $1$ is total preservation, and $0.1$ means it's being compressed by a factor of 10. 

In [ ]:
#|export
from autometric.metrics import PullbackMetric
import numpy as np
import torch
def determinants_of_encoder_pullback(model, dataloader):
    # returns the determinants of the metric matrices for each point in the dataset
    Metric = PullbackMetric(model.input_dim, model.encoder)
    Gs = Metric.metric_matrix(dataloader.dataset.pointcloud).detach().cpu().numpy()
    dets = [np.linalg.det(G) for G in Gs]
    return np.array(dets)

INFO: Using pytorch backend


In [ ]:
#|export
from autometric.metrics import PullbackMetric
import numpy as np
import torch
def trace_of_encoder_pullback(model, dataloader):
    # returns the determinants of the metric matrices for each point in the dataset
    Metric = PullbackMetric(model.input_dim, model.encoder)
    Gs = Metric.metric_matrix(dataloader.dataset.pointcloud).detach().cpu().numpy()
    dets = [np.sum(np.linalg.eigvals(G)) for G in Gs]
    return np.array(dets)

In [ ]:
#|export
from autometric.metrics import PullbackMetric
import numpy as np
import torch
def rank_of_encoder_pullback(model, dataloader, eps=1e-10):
    # returns the determinants of the metric matrices for each point in the dataset
    Metric = PullbackMetric(model.input_dim, model.encoder)
    Gs = Metric.metric_matrix(dataloader.dataset.pointcloud).detach().cpu().numpy()
    ranks = [np.sum((np.linalg.eigvals(G)>eps).astype(int)) for G in Gs]
    return np.array(ranks)

In [ ]:
#|export
from autometric.metrics import PullbackMetric
import numpy as np
import torch
def spectral_entropy_of_matrix(A):
    # returns the spectral entropy of a matrix
    # A is a numpy array
    eigvals = np.linalg.eigvals(A)
    eigvals = eigvals[eigvals > 0]
    eigvals /= eigvals.sum()
    return -np.sum(eigvals * np.log(eigvals))

def spectral_entropy_of_encoder_pullback(model, dataloader):
    # returns the determinants of the metric matrices for each point in the dataset
    Metric = PullbackMetric(model.input_dim, model.encoder)
    Gs = Metric.metric_matrix(dataloader.dataset.pointcloud).detach().cpu().numpy()
    entropies = [spectral_entropy_of_matrix(G) for G in Gs]
    return np.array(entropies)

In [ ]:
#|export
def evals_of_encoder_pullback(model, dataloader):
    # returns the determinants of the metric matrices for each point in the dataset
    Metric = PullbackMetric(model.input_dim, model.encoder)
    Gs = Metric.metric_matrix(dataloader.dataset.pointcloud).detach().cpu().numpy()
    e = [np.sort(np.linalg.eigvals(G))[::-1] for G in Gs]
    return np.vstack(e)

In [ ]:
#|export
def smallest_eigenvector(matrix):
    """
    Find the eigenvector associated with the smallest eigenvalue of a square matrix.

    Args:
        matrix (np.array): A square numpy array representing a matrix.

    Returns:
        np.array: The eigenvector associated with the smallest eigenvalue.
    """
    # Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(matrix)

    # Find the index of the smallest eigenvalue
    min_index = np.argmin(eigenvalues)

    # Return the corresponding eigenvector
    return eigenvectors[:, min_index]
def normal_vectors_of_encoder_pullback(model, dataloader):
    # returns the determinants of the metric matrices for each point in the dataset
    Metric = PullbackMetric(model.input_dim, model.encoder)
    Gs = Metric.metric_matrix(dataloader.dataset.pointcloud).detach().cpu().numpy()
    e = [smallest_eigenvector(G) for G in Gs]
    return np.vstack(e)

We'll wrap all of these metrics into a convenient form so they can be run en-masse on a single model, without having to specify each of them.

In [ ]:
#|export
import matplotlib.pyplot as plt
import numpy
from autometric.utils import *
from mpl_toolkits.mplot3d import Axes3D

def visualize_encoder_pullback_metrics(model, dataloader, title):
    X = model.encoder(dataloader.dataset.pointcloud).cpu().detach().numpy()
    fig, axs = plt.subplots(2, 3, figsize=(12, 8))

    spectral_entropy = spectral_entropy_of_encoder_pullback(model,dataloader)
    axs[0,0].scatter(X[:,0],X[:,1],c=spectral_entropy)
    axs[0,0].set_title("Spectral Entropy")

    trace = trace_of_encoder_pullback(model,dataloader)
    axs[0,1].scatter(X[:,0],X[:,1],c=trace)
    axs[0,1].set_title("Trace")

    rank = rank_of_encoder_pullback(model,dataloader)
    axs[0,2].scatter(X[:,0],X[:,1],c=rank)
    axs[0,2].set_title("Rank")
    
    evals = evals_of_encoder_pullback(model, dataloader)
    for i in range(3):
        axs[1,i].scatter(X[:,0],X[:,1],c=evals[:,i])
        axs[1,i].set_title(f"{printnum(i)} Eigenvalue")
    
    fig.suptitle(title)
    plt.tight_layout()
    plt.show()

In [ ]:
#|export
import matplotlib.pyplot as plt
import numpy
from autometric.utils import *
from mpl_toolkits.mplot3d import Axes3D

def visualize_encoder_pullback_metrics_in_ambient_space(model, dataloader, title):
    X = model.encoder(dataloader.dataset.pointcloud).cpu().detach().numpy()
    D = dataloader.dataset.pointcloud.cpu().detach().numpy()
    figure = plt.figure()

    ax = figure.add_subplot(231, projection='3d')
    spectral_entropy = spectral_entropy_of_encoder_pullback(model,dataloader)
    ax.scatter(D[:,0],D[:,1],D[:,2],c=spectral_entropy)
    ax.set_title("Spectral Entropy")

    ax = figure.add_subplot(232, projection='3d')
    trace = trace_of_encoder_pullback(model,dataloader)
    ax.scatter(D[:,0],D[:,1],D[:,2],c=trace)
    ax.set_title("Trace")

    ax = figure.add_subplot(233, projection='3d')
    rank = rank_of_encoder_pullback(model,dataloader)
    ax.scatter(D[:,0],D[:,1],D[:,2],c=rank)
    ax.set_title("Rank")
    
    evals = evals_of_encoder_pullback(model, dataloader)
    for i in range(3):
        ax = figure.add_subplot(230+i+4, projection='3d')
        ax.scatter(D[:,0],D[:,1],D[:,2],c=evals[:,i])
        ax.set_title(f"{printnum(i)} Eigenvalue")
    
    figure.suptitle(title)
    plt.show()

In [ ]:
%notebook save
!nbdev_export